In [42]:
# External modules
import os.path
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import time


# Internal modules
import results
from param_input import V_meta, V_bounds, P, P_meta, Days, Hours, Periods
from param_calc import Periods, Irradiance, Ext_T, Dates
import model
import plot
from plot import fig_width, fig_height

In [43]:
objective = 'emissions'
m = model.run(objective)

Reset all parameters
Discarded solution information
Changed value of parameter TimeLimit to 2000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
******************************************************************
******************* Objective: emissions **************************
******************************************************************
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 6106 rows, 5955 columns and 17011 nonzeros
Model fingerprint: 0x963c1cd0
Model has 146 general constraints
Variable types: 5373 continuous, 582 integer (582 binary)
Coefficient statistics:
  Matrix range     [3e-05, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+04]
  RHS range        [5e-05, 1e+04]
Presolve removed 3387 rows and 2376 columns
Presolve time: 0.03s
Presolved: 2719 rows, 3579 columns, 9356 nonzeros
Variable types: 3296 continuous, 283 integer (283 binary)

Root relaxation: objective 1.033319e+01, 3080 iterations, 0.08 seconds

    

In [79]:
var_names = set()
last_cahr = set()
for v in m.getVars():
    name = re.split('\d', v.VarName)[0]
    var_names.add(name)
    last_cahr.add(name[-1])
var_names

{'build_T[',
 'build_cons_Heat[',
 'capex',
 'comfort_delta_T[',
 'comfort_delta_T_abs[',
 'emissions',
 'grid_export[Elec,',
 'grid_export[Gas,',
 'grid_export_a[Elec]',
 'grid_export_a[Gas]',
 'grid_import[Elec,',
 'grid_import[Gas,',
 'grid_import_a[Elec]',
 'grid_import_a[Gas]',
 'heat_loss_biomass[',
 'max_T_AD',
 'min_T_AD',
 'opex',
 'penalty[',
 'penalty_a',
 'totex',
 'unit_SOC[BAT][',
 'unit_SOC[CGT][',
 'unit_T[AD][',
 'unit_capex[AD]',
 'unit_capex[BAT]',
 'unit_capex[BOI]',
 'unit_capex[CGT]',
 'unit_capex[PV]',
 'unit_capex[SOFC]',
 'unit_charge[BAT][',
 'unit_charge[CGT][',
 'unit_cons[AD][Biomass,',
 'unit_cons[AD][Elec,',
 'unit_cons[AD][Heat,',
 'unit_cons[BAT][Elec,',
 'unit_cons[BOI][Biogas,',
 'unit_cons[BOI][Gas,',
 'unit_cons[CGT][Biogas,',
 'unit_cons[CGT][Elec,',
 'unit_cons[SOFC][Biogas,',
 'unit_cons[SOFC][Gas,',
 'unit_discharge[BAT][',
 'unit_discharge[CGT][',
 'unit_install[AD]',
 'unit_install[BAT]',
 'unit_install[BOI]',
 'unit_install[CGT]',
 'unit_inst

In [45]:
last_cahr

{',', 'D', '[', ']', 'a', 's', 'x'}

In [46]:
for v in m.getVars():
    if '100' in v.varName:
        print(v.varName)

unit_prod[CGT][Biogas,100]
unit_cons[CGT][Biogas,100]
unit_cons[CGT][Elec,100]
unit_SOC[CGT][100]
unit_charge[CGT][100]
unit_discharge[CGT][100]


In [57]:
var_results = {}
var_names = set()
for v in m.getVars():
    name = re.split('\d', v.VarName)[0]
    Time_steps = 'Daily' if 'CGT' not in name else 'Annual'
    if name[-1] == ',':
        name = name[:-1] + ']'
    if name[-1] == '[':
        name = name[:-1]
    var_names.add(name)
    if name not in var_results.keys():
        var_results[name] = [v.x]
    else:
        var_results[name].append(v.x)
var_results

{'unit_prod[BOI][Heat]': [454.3563061930749,
  453.8482004384097,
  463.5022097770778,
  474.17243062507805,
  489.92370901974397,
  432.0533499730919,
  277.260084750915,
  209.64620364337517,
  148.97364056160586,
  184.8178296060449,
  294.62988726348493,
  320.40108773842917,
  165.8084194477833,
  210.6499761695829,
  176.81485113670826,
  156.23313662027317,
  189.42938517404613,
  254.39532579091332,
  405.1237449757605,
  434.04997500152626,
  436.0823980201944,
  435.5742922655292,
  452.32388317441325,
  457.40494072107987,
  454.8644119477466,
  466.5508443050772,
  490.4318147744101,
  497.03718958507966,
  513.2965737344117,
  476.7666563837583,
  332.6436120095807,
  273.97046199538033,
  212.99496564961024,
  197.1499836007141,
  315.61914097548197,
  335.2737705064317,
  198.38634474378583,
  219.92384469224504,
  161.65138871537798,
  221.16326150027513,
  282.4127382780435,
  353.98405370558027,
  500.13952109842603,
  540.7521834815288,
  558.5358848948573,
  572.762

In [90]:
var_results = {}
for n in var_names:
    Time_steps = 'Daily' if 'CGT' not in n else 'Annual'
    if n[-1] == ',' or n[-1] == '[':
        results = []
        if Time_steps == 'Daily':
            results = [[m.getVarByName(n + f'{d},{h}]').x for h in Hours] for d in Days]
        elif Time_steps == 'Annual':
            results = [m.getVarByName(n + f'{p}]').x for p in Periods]
        n = n[:-1] + ']'
        var_results[n] = results
    else:
        var_results[n] = m.getVarByName(n).x
var_results

{'unused[Biogas]': [[0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  [0.0,
   0.0,
 

In [164]:
def all_results(m, Days, Hours, Periods):
    var_names_cut = set()
    for v in m.getVars():
        name = re.split('\d', v.VarName)[0]
        var_names_cut.add(name)
    
    var_results, var_bounds = ({'time_indep': {}, 'time_dep':{}} for i in range(2))
    for n in var_names_cut:
        results = []
        Time_steps = 'Daily' if 'CGT' not in n else 'Annual'
        if n[-1] == ',' or n[-1] == '[':
            key = 'time_dep'
            if Time_steps == 'Daily':
                results = [[m.getVarByName(n + f'{d},{h}]').x for h in Hours] for d in Days]
                x = lambda v: [v.lb, v.ub]
                bounds = x(m.getVarByName(n + '0,0]'))
            elif Time_steps == 'Annual':
                results = [m.getVarByName(n + f'{p}]').x for p in Periods]
            n = n[:-1] + ']' if n[-1] == ',' else n[:-1]
        else:
            key = 'time_indep'
            results = m.getVarByName(n).x
        var_bounds[key][n] = bounds
        var_results[key][n] = results
    return var_results, var_bounds

In [165]:
var_results, var_bounds = all_results(m, Days, Hours, Periods)
var_names = {'time_indep': None, 'time_dep': None}
for t in ['time_indep', 'time_dep']:
    var_names[t] = list(var_results[t].keys())
    var_names[t].sort()

In [167]:
var_bounds['time_indep']['max_T_AD']

[0.0, 10000.0]

In [72]:
for d,h in iterate:
    print(d,h)

TypeError: 'function' object is not iterable

In [69]:
Hours

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23]